# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,-9.53,82,99,2.02,RU,1743560437
1,1,kurduvadi,18.0833,75.4333,28.69,39,63,6.13,IN,1743560438
2,2,portland,45.5234,-122.6762,11.56,79,100,1.54,US,1743560015
3,3,blackmans bay,-43.0167,147.3167,17.07,58,13,0.45,AU,1743560441
4,4,mwaro,-3.5000,29.6667,12.95,92,100,0.64,BI,1743560442


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",          # Longitude on x-axis
    "Lat",          # Latitude on y-axis
    size="Humidity",# humidity for point size
    geo=True,       # geographic plotting
    tiles="OSM",    # OpenStreetMap as background
    color="City",   # color for each city
    alpha=0.6,      # point transparency
    frame_width=800,
    frame_height=500,
    legend=True
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= 21) &
    (city_data_df["Max Temp"] <= 27) &
    (city_data_df["Cloudiness"] == 0) &
    (city_data_df["Wind Speed"] < 4)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,broken hill,-31.9500,141.4333,25.35,28,0,3.97,AU,1743560451
35,35,veraval,20.9000,70.3667,26.03,39,0,2.84,IN,1743560477
109,109,morondava,-20.2833,44.2833,25.14,74,0,1.85,MG,1743560565
112,112,timimoun,29.2639,0.2310,21.21,14,0,3.60,DZ,1743560470
204,204,rotonda west,26.8837,-82.2901,24.47,96,0,0.00,US,1743560677


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,broken hill,AU,-31.9500,141.4333,28,
35,veraval,IN,20.9000,70.3667,39,
109,morondava,MG,-20.2833,44.2833,74,
112,timimoun,DZ,29.2639,0.2310,14,
204,rotonda west,US,26.8837,-82.2901,96,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 #meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
broken hill - nearest hotel: Ibis Styles
veraval - nearest hotel: Shyam
morondava - nearest hotel: Hotel Sharon Menabe
timimoun - nearest hotel: أناف
rotonda west - nearest hotel: Sea Oats Beach Club
turabah - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
jawf al maqbabah - nearest hotel: فندق لودر
umarkot - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
salumbar - nearest hotel: Swagat Guest House
beroroha - nearest hotel: No hotel found
rio de janeiro - nearest hotel: Ibis Rio porto atlantico


,City,Country,Lat,Lng,Humidity,Hotel Name
12,broken hill,AU,-31.9500,141.4333,28,Ibis Styles
35,veraval,IN,20.9000,70.3667,39,Shyam
109,morondava,MG,-20.2833,44.2833,74,Hotel Sharon Menabe
112,timimoun,DZ,29.2639,0.2310,14,أناف
204,rotonda west,US,26.8837,-82.2901,96,Sea Oats Beach Club
280,turabah,SA,21.2141,41.6331,22,No hotel found
310,toliara,MG,-23.3500,43.6667,69,Ambary
326,jawf al maqbabah,YE,13.8378,45.8349,59,فندق لودر
379,umarkot,PK,25.3616,69.7362,13,No hotel found
409,salalah,OM,17.0151,54.0924,60,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.7,
    hover_cols=["City", "Country", "Hotel Name", "Humidity"],
    frame_width=800,
    frame_height=500
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)